In [4]:
import spacy
import os
import nltk
from current_project_programs.preprocess import Preprocess
from nltk.parse.stanford import StanfordParser
from nltk.corpus import stopwords
from spacy import displacy
from spacy.matcher import Matcher
import json
from spacy.tokens import Doc, Span, Token
import neuralcoref
import time
from nltk.corpus import wordnet
import pickle

def load_text_files(filename, dir='dataset/'):
    list_of_directory = os.listdir(dir)
    dataset = {}
    test_sentences = []
    #test_sentences = ["XYZ acquired PQR","In 2017, Amazon acquired Whole Foods Market for US$13.4 billion, which vastly increased Amazon's presence as a brick-and-mortar retailer.", 'In January 2015, Amazon Web Services acquired Annapurna Labs, an Israel-based microelectronics company reputedly for US$350â\x80\x93370M.', 'Amazon acquired Junglee in 1998, and the website Junglee.com was launched in India in February 2012 as a comparison-shopping website.', 'Later that month, Twitch was acquired by Amazon for $970 million.', "Next, Apple successfully acquired the German company Astarte, which had developed DVD authoring technology, as well as Astarte's corresponding products and engineering team in April 2000.", "In July 2001, Apple acquired Spruce Technologies, a PC DVD authoring platform, to incorporate their technology into Apple's expanding portfolio of digital video projects.", 'In May 2014, the company confirmed its intent to acquire Dr. Dre and Jimmy Iovine\'s audio company Beats Electronicsâ\x80\x94producer of the "Beats by Dr. Dre" line of headphones and speaker products, and operator of the music streaming service Beats Musicâ\x80\x94for $3 billion, and to sell their products through Apple\'s retail outlets and resellers.', 'On February 14, 2019, Apple acquired DataTiger for its digital marketing technology.', 'It features the Apple A5 processor and Siri voice assistant technology, the latter of which Apple had acquired in 2010.', 'In 2006, AT&T Inc. acquired BellSouth, the last independent Baby Bell company, making their formerly joint venture Cingular Wireless (which had acquired AT&T Wireless in 2004) wholly owned and rebranding it as AT&T Mobility.', "One of that company's subsidiaries was American Telephone and Telegraph Company (AT&T), established in 1885, which acquired the Bell Company on December 31, 1899, for legal reasons, leaving AT&T as the main company.", 'In the later half of the 1990s, the company acquired several other telecommunications companies, including some Baby Bells, while selling its cable business.', 'In order for AT&T to fully acquire Time Warner, the Department of Justice stated that the company must divest either DirecTV or Turner Broadcasting System.', 'In October 2016, AT&T announced a deal to acquire Time Warner worth $85.4 billion (including assumed Time Warner debt).', "The proposed deal would give AT&T significant holdings in the media industry; AT&T's competitor Comcast had previously acquired NBCUniversal in a similar bid to increase its media holdings, in concert with its ownership of television and internet providers.", 'Ameritech, acquired by SBC in 1999', 'AT&T Corp., acquired by SBC in 2005', 'BellSouth, acquired by AT&T in 2006', 'Pacific Telesis, acquired by SBC in 1997', 'Southwestern Bell, rebranded as SBC Communications in 1995, acquired AT&T Corporation and rebranded as AT&T Inc. in 2005', 'The company is also active in Mexico, and it was announced on November 7, 2014, that Mexican carrier Iusacell is being acquired by AT&T.', 'On April 30, 2015, AT&T acquired wireless operations Nextel Mexico from NII Holdings (now AT&T Mexico).', 'Since 2016, the company has acquired large holdings in the major US airline carriers, and is currently the largest shareholder in United Airlines and Delta Air Lines, and a top three shareholder in Southwest Airlines and American Airlines.', "In the late 1970s, Berkshire acquired an equity stake in the Government Employees Insurance Company (GEICO), which forms the core of its insurance operations today (and is a major source of capital for Berkshire Hathaway's other investments).", "Despite its size, Berkshire had for many years not been included in broad stock market indices such as the S&P 500 due to the lack of liquidity in its shares; however, following a 50-to-1 split of Berkshire's class B shares in January 2010, and Berkshire's announcement that it would acquire the Burlington Northern Santa Fe Corporation, parent of BNSF Railway, Berkshire replaced BNSF in the S&P 500 on February 16, 2010.", 'GEICO â\x80\x93 Berkshire acquired GEICO in January 1996.', 'Gen Re â\x80\x93 Berkshire acquired General Re in December 1998.', 'NRG (Nederlandse Reassurantie Groep) â\x80\x93 Berkshire acquired NRG, a Dutch life reinsurance company, from ING Group in December 2007.', 'Berkshire acquired Fruit of the Loom on April 29, 2002 for $835 million in cash.', 'Berkshire acquired Russell Corporation on August 2, 2006 for $600 million or $18.00 per share.', 'It expanded its building products business in December 2000, when it acquired Benjamin Moore & Co. of Montvale, New Jersey.', 'In 2001, Berkshire acquired three additional building products companies.', 'In July, Berkshire acquired a 90% equity interest in MiTek Inc., which makes engineered connector products, engineering software and services, and manufacturing machinery for the truss fabrication segment of the building components industry and is headquartered in Chesterfield, Missouri.', "Finally in 2001, Berkshire acquired 87 percent of Dalton, Georgia-based Shaw Industries, Inc. Shaw is the world's largest carpet manufacturer based on both revenue and volume of production and designs and manufactures over 3,000 styles of tufted and woven carpet and laminate flooring for residential and commercial use under approximately 30 brand and trade names and under certain private labels.", 'On August 7, 2003, Berkshire acquired Clayton Homes, Inc. Clayton, headquartered near Knoxville, Tennessee, is a vertically integrated manufactured housing company.', 'On May 1, 2008, Mitek acquired Hohmann & Barnard, a fabricator of anchors and reinforcement systems for masonry and on October 3 of that year, Mitek acquired Blok-Lok, Ltd. of Toronto, Canada.', 'On April 23, 2010, Mitek acquired the assets of Dur-O-Wal from Dayton Superior.', 'In 1996, Berkshire acquired FlightSafety International Inc. (or FSI), founded in 1951 by Albert Lee Ueltschi.', 'Berkshire acquired NetJets Inc. in 1998.', 'The home furnishings businesses are the Nebraska Furniture Mart, RC Willey Home Furnishings, Star Furniture Company, and Jordan\'s Furniture, Inc. CORT Business Services Corporation was acquired in 2000 by an 80.1% owned subsidiary of Berkshire and is the leading national provider of rental furniture, accessories and related services in the "rent-to-rent" segment of the furniture rental industry.', 'In 2002, Berkshire acquired The Pampered Chef, Ltd., the largest direct seller of kitchen tools in the United States.', 'In November 2012, Berkshire announced they would acquire the Oriental Trading Company, a direct marketing company for novelty items, small toys, and party items.', 'On October 3, 2017, it was announced that Berkshire Hathaway will acquire 38.6% of truck stop chain Pilot Flying J, with plans to increase its stake to 80% in 2023.', "In late June 2017, Berkshire Hathaway indirectly acquired $400 million of the Toronto-based company's common shares as he gave a lifeline to the embattled mortgage lender Home Capital Group Inc.", 'In 2002, Berkshire acquired Albecca Inc. Albecca is headquartered in Norcross, Georgia, and primarily does business under the Larson-Juhl name.', 'Berkshire acquired CTB International Corp. in 2002.', 'Berkshire acquired McLane Company, Inc. in May 2003 from Walmart, which brought on other subsidiaries such as Professional Datasolutions, Inc. and Salado Sales, among others.', 'Berkshire acquired McLane Company, Inc. in May 2003 from Walmart, which brought on other subsidiaries such as Professional Datasolutions, Inc. and Salado Sales, among others.', 'On December 25, 2007, Berkshire Hathaway acquired Marmon Group.', 'On November 14, 2014, Berkshire Hathaway announced that it would acquire Duracell from Procter & Gamble for $4.7 billion in an all-stock deal.', 'Berkshire acquired Clayton Homes, a maker of modular homes, storage trailers, chassis, intermodal piggyback trailers and domestic containers.', "In 2006 Berkshire Hathaway Inc. acquired Russell Corporation for $600 million, in fact getting most shares and brands in many sports leagues â\x80\x93 including Spalding NBA official basketballs, BIKE Athletic Company protections, AAI (American Athletic) Gymnastics' tables, bars, rings, horses or Dudley softball balls and accessories.", 'On November 3, 2009, Berkshire Hathaway announced that, using stock and cash totaling $26 billion, it would acquire the remaining 77.4 percent of the Burlington Northern Santa Fe Corporation, parent of BNSF Railway, that it did not already own.', 'On March 14, 2011, Berkshire Hathaway announced that it would acquire the Lubrizol Corporation for $9 billion in cash, a deal that was described as one of the largest deals ever for Berkshire Hathaway.', 'The company became a majority owner of Heinz on June 18, 2015, after exercising a warrant to acquire 46,195,652 shares of common stock for a total price of $461,956.52 increasing its stake to 52.5%.', 'On August 10, 2015, the boards of directors of Berkshire Hathaway Inc. and Precision Castparts Corp. unanimously approved a definitive agreement for Berkshire Hathaway to acquire, for $235 per share in cash, all outstanding PCC shares.', 'In May 2016, it was revealed in a regulatory filing that Berkshire had acquired a stake in Apple Inc.', 'Meanwhile, the distinctive Travelers red umbrella logo, which was also acquired in the deal, was applied to all the businesses within the newly named organization.', 'During this period, Travelers acquired Shearson Lehmanâ\x80\x94a retail brokerage and asset management firm that was headed by Weill until 1985â\x80\x94and merged it with Smith Barney.', 'In November 1997, Travelers Group (which had been renamed again in April 1995 when they merged with Aetna Property and Casualty, Inc.), acquired Salomon Brothers, a major bond dealer and bulge bracket investment bank, in a $9 billion transaction.', 'In the transaction, Travelers Group acquired all Citicorp shares for $70 billion in stock, issuing 2.5 new Citigroup shares for each Citicorp share.', 'In 2000, Citigroup acquired Associates First Capital Corporation for $31.1 billion in stock, which, until 1989, had been owned by Gulf+Western (now part of National Amusements), and later by Ford Motor Credit Company.', 'In return the bank gave the U.S. Treasury $27 billion of preferred shares and warrants to acquire common stock.', 'In November 2015, Springleaf acquired OneMain Financial from Citigroup.', 'In 1898, Standard Oil of New Jersey acquired controlling stake in Imperial Oil of Canada.', 'In 1919, Jersey Standard acquired a 50% share in Humble Oil & Refining Co., a Texas oil producer.', 'In the following years it acquired or established Tropical Oil Company of Colombia (1920), Standard Oil Company of Venezuela (1921), and Creole Petroleum Company of Venezuela (1928).', 'In 1932, Jersey Standard acquired foreign assets of the Pan American Petroleum and Transport Company.', 'In 1948, Jersey Standard and Socony-Vacuum acquired interests in the Arab-American Oil Company (Aramco).', 'In 1965, Jersey Standard started to acquire coal assets through its affiliate Carter Oil (later renamed Exxon Coal, U.S.A.).', 'In 1967, Mobil acquired a 28% strategic stake in the German fuel chain Aral.', 'In 1973, Exxon acquired the Ray Point uranium ore processing facility which was shortly afterwards decommissioned.', "As a result, BP acquired all fuels assets, two base oil plants, and a substantial part of the joint venture's finished lubricants business, while ExxonMobil acquired other base oil plants and a part of the finished lubricants business.", 'The stake in Aral was sold to Vega Oel, later acquired by BP.', 'The stake in Aral was sold to Vega Oel, later acquired by BP.', 'MEGAS was acquired by Duke Energy and the stake in Thyssengas was acquired by RWE.', 'By this time, as a result of numerous mergers and rebranding, the remaining Standard Oil companies that previously objected to the Esso name had been acquired by BP.', 'In 2002, IBM acquired PwC consulting, and in 2003 it initiated a project to redefine company values, hosting a three-day online discussion of key business issues with 50,000 employees.', "In 2005, the company sold its personal computer business to Chinese technology company Lenovo and, in 2009, it acquired software company SPSS Inc. Later in 2009, IBM's Blue Gene supercomputing program was awarded the National Medal of Technology and Innovation by U.S. President Barack Obama.", "In 2005, the company sold its personal computer business to Chinese technology company Lenovo and, in 2009, it acquired software company SPSS Inc. Later in 2009, IBM's Blue Gene supercomputing program was awarded the National Medal of Technology and Innovation by U.S. President Barack Obama.", 'In 2012, IBM announced it has agreed to buy Kenexa, and a year later it also acquired SoftLayer Technologies, a web hosting service, in a deal worth around $2 billion.', 'In 2012, IBM announced it has agreed to buy Kenexa, and a year later it also acquired SoftLayer Technologies, a web hosting service, in a deal worth around $2 billion.', 'In 2016, IBM acquired video conferencing service Ustream and formed a new cloud video unit.', 'In October 2018, IBM announced its intention to acquire Red Hat for $34 billion.', 'Weakened by the real estate collapse in the early 1990s, it was acquired by Chemical Bank in 1996, retaining the Chase name.', 'In 1999, it acquired San Francisco-based Hambrecht & Quist for $1.35 billion.', 'In 1996, Chemical Bank acquired Chase Manhattan.', 'With the beginning of interstate banking they spread into other states, always renaming acquired banks "Bank One", though for a long time they resisted combining them into one bank.', 'On March 16, 2008, after a weekend of intense negotiations between JPMorgan, Bear, and the federal government, JPMorgan Chase announced that it had plans to acquire Bear Stearns in a stock swap worth $2.00 per share or $240 million pending shareholder approval scheduled within 90 days.', 'Under the revised terms, JPMorgan also immediately acquired a 39.5% stake in Bear Stearns (using newly issued shares) at the new offer price and gained a commitment from the board (representing another 10% of the share capital) that its members would vote in favor of the new deal.', 'Through the acquisition, JPMorgan now owns the former accounts of Providian Financial, a credit card issuer WaMu acquired in 2005.', "In April 2006, JPMorgan Chase acquired Bank of New York Mellon's retail and small business banking network.", 'In March 2008, JPMorgan acquired the UK-based carbon offsetting company ClimateCare.', 'In November 2009, JPMorgan announced it would acquire the balance of JPMorgan Cazenove, an advisory and underwriting joint venture established in 2004 with the Cazenove Group, for GBP1 billion.', 'In January 2013, JPMorgan acquired Bloomspot, a San Francisco-based startup in the "deals" space for $35 million.', 'Bernie Madoff opened a business account at Chemical Bank in 1986 and maintained it until 2008, long after Chemical acquired Chase.', 'In 2015, Tesla acquired Riviera Tool & Die (with 100 employees in Michigan), one of its suppliers of stamping items.', 'In 2017, Tesla acquired Perbix Machine Company, a manufacturer of automated manufacturing equipment, that had been an equipment supplier for over three years.', 'In late 2016, Tesla acquired German engineering firm Grohmann Engineering in PrÃ¼m as a new division dedicated to helping Tesla increase the automation and effectiveness of its manufacturing process.', 'Riviera Tool LLC was acquired for an unknown price.', 'In August 2013, the firm announced it was in talks to acquire a majority stake in the Kenya-based supermarket chain, Naivas.', "On August 8, 2016, Walmart announced a deal to acquire e-commerce website Jet.com for US$3.3 billion Jet.com co-founder and CEO Marc Lore stayed on to run Jet.com in addition to Walmart's existing U.S. e-commerce operation.", "On June 16, 2017, Walmart agreed to acquire the men's apparel company Bonobos for $310 million in an effort to expand its fashion holdings.", 'On September 29, 2017, Walmart acquired Parcel, a technology-based, same-day and last-mile delivery company in Brooklyn.', 'On 26 February 2019, Walmart announced that it had acquired Tel Aviv-based product review start-up Aspectiva for an undisclosed sum.', 'In January 2009, the company acquired a controlling interest in the largest grocer in Chile, DistribuciÃ³n y Servicio D&S SA.', 'Walmart has operated in Canada since it acquired 122 stores comprising the Woolco division of Woolworth Canada, Inc in 1994.', "Walmart's UK subsidiary Asda (which retained its name after being acquired by Walmart) is based in Leeds and accounted for 42.7 percent of 2006 sales of Walmart's international division.", 'On May 9, 2018, Walmart announced its intent to acquire a 77% majority stake in the Indian e-commerce company Flipkart for $16 billion, in a deal that was completed on August 18, 2018.', 'In 1997, Walmart took over the supermarket chain Wertkauf with its 21 stores for DM 750 million and the following year Walmart acquired 74 Interspar stores for DM 1.3 billion.', 'Jet.com has acquired its own share of online retailers such as Hayneedle in March 2016, Shoebuy.com in December 2016, and ModCloth in March 2017.', 'Jet.com has acquired its own share of online retailers such as Hayneedle in March 2016, Shoebuy.com in December 2016, and ModCloth in March 2017.', 'Walmart also acquired Parcel, a delivery service in New York, on September 29, 2017.', 'On February 15, 2017, Walmart acquired Moosejaw, an online active outdoor retailer, for approximately $51 million.', 'On May 9, 2018, Walmart announced its intent to acquire a 77% controlling stake in the Indian e-commerce website Flipkart for $16 billion (beating bids by Amazon.com), subject to regulatory approval.', 'In April 2011, Walmart acquired Kosmix to develop software for analyzing real-time data streams.']
    for file in list_of_directory:
        if (not filename and file.endswith('.txt')) or file.endswith(filename):
            current = open(dir+file,"r",encoding='ISO-8859-1')
            list_of_paragraphs = []
            for count, line in enumerate(current):
                list_of_paragraphs.append(line.strip().replace('ï»¿','').replace('â','-'))
            dataset[file] = list_of_paragraphs
    return dataset

def feature_extraction(word):
    definition,synonyms,antonyms,hyper,hypo,mero,holo = [],[],[],[],[],[],[]
    for syn in wordnet.synsets(word):
        definition.append(syn.definition())
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
            if lemma.antonyms():
                antonyms.append(lemma.antonyms()[0].name())
        for hypernym in syn.hypernyms()[:2]:
            hyper.append(hypernym.lemma_names())
        for hyponym in syn.hyponyms()[:2]:
            hypo.append(hyponym.lemma_names())
        for meronym in syn.part_meronyms()[:2]:
            mero.append(meronym.lemma_names())
        for holonym in syn.part_holonyms()[:2]:
            holo.append(holonym.lemma_names())
    return definition,synonyms,antonyms,hyper,hypo,mero,holo

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    if doc[start].ent_iob_=='B' and (len(doc)==end or doc[end].ent_type_!='GPE'):
        sent = span.sent  # Sentence containing matched span
        extraction = {"template":"PART", "sentences":[sent.text]}
        arguments = {}
        # Append mock entity for match in displaCy style to matched_sents
        # get the match span by ofsetting the start and end of the span with the
        # start and end of the sentence in the doc
        match_ents = [{
            "start": span.start_char - sent.start_char,
            "end": span.end_char - sent.start_char,
            "label": "PLACE_IN_PLACE",
        }]
        matched_sents.append({"text": sent.text, "ents": match_ents})
        arguments["1"], arguments["2"] = ''.join(str(span)).split(',')
        extraction["arguments"] = arguments
        result["extraction"].append(extraction)
        
def show_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    sent = span.sent  # Sentence containing matched span
    # Append mock entity for match in displaCy style to matched_sents
    # get the match span by ofsetting the start and end of the span with the
    # start and end of the sentence in the doc
    match_ents = [{
        "start": span.start_char - sent.start_char,
        "end": span.end_char - sent.start_char,
        "label": "PLACE",
    }]
    matched_sents.append({"text": sent.text, "ents": match_ents})
    
def merge_ents(doc):
    #print_token_attr(doc)
    with doc.retokenize() as retokenizer:
        for ent in doc.ents:
            retokenizer.merge(ent)
            
    i = 0
    while i<len(doc):
        token = doc[i]
        if token.pos_=='PROPN':
            if token.dep_=='compound':
                tail = token
                while tail.i+1<len(doc) and tail.nbor().dep_=='compound':
                    tail = tail.nbor()
                if tail.i>token.i:
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc[token.i:tail.i+1])
        i += 1
        
    i = 0    
    while i<len(doc):
        token = doc[i]
        if token.pos_=='PROPN':
            if token.dep_ == 'nmod':
                if token.head.lemma_.lower() in place_lemmas:
                    place_type = token.head
                    arr = [x.text.strip()+' '+place_type.lemma_ for x in doc[token.i:place_type.i+1] if x.pos_=='PROPN' and x.lemma_.lower() not in place_lemmas]
                    if not (place_type.i+1<len(doc) and place_type.nbor().lemma_.lower()=='of'):
                        with doc.retokenize() as retokenizer:
                            retokenizer.merge(doc[token.i:place_type.i+1], attrs={"ENT_TYPE":'GPE'})
                        if len(arr)>1:
                            doc[token.i]._.grouping_type = "SIBLING"
                            doc[token.i]._.list = arr
        i += 1
    return doc

def merge_siblings(doc):
    #print_token_attr(doc)
    i = 0
    while i<len(doc):
        token = doc[i]
        if token.pos_=='CCONJ':
            conj = token.text
            a = token.head.i
            b = token.i
            while b<len(doc) and doc[b].text != [x for x in token.head.children][-1].text:
                b += 1                
            cur = token.head
            ent_type = {}
            ent_type[doc[b].ent_type_] = 1
            if cur.ent_type_ not in ent_type:
                ent_type[cur.ent_type_] = 1
            else:
                ent_type[cur.ent_type_] += 1                
            while cur.head.pos_=='PROPN':
                cur = cur.head
                if cur.ent_type_ not in ent_type:
                    ent_type[cur.ent_type_] = 1
                else:
                    ent_type[cur.ent_type_] += 1
                a = cur.i
                if cur.head == cur:
                    break            
            mod = ''
            attr = {'ENT_TYPE': None}
            if doc[a].dep_=='nmod':
                mod = doc[a].head.lemma_
                if mod.lower() in place_lemmas:
                    attr = {"ENT_TYPE": 'GPE'}
            else:
                attr = {"ENT_TYPE": [k for k, v in sorted(ent_type.items(), key=lambda item: item[1])][-1]}
            if attr["ENT_TYPE"] == 'GPE':
                with doc.retokenize() as retokenizer:
                    retokenizer.merge(doc[a:b+1], attrs=attr)
                doc[a]._.grouping_type = "SIBLING"
                doc[a]._.list = [y.strip()+' '+mod for x in doc[a].text.split(conj) for y in x.split(',')]
                i = a
                #print(doc[a:b+1], attrs)
        i += 1
    return doc

def merge_ancestors(doc):
    #print_token_attr(doc)
    i = 0
    while i<len(doc):
        token  = doc[i]
        if token.ent_type_=='GPE':
            if token.dep_ == 'poss':
                if token.head.ent_type_=='GPE':
                    parent = token.text.strip()
                    child = token.head.text.strip()
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc[token.i:token.head.i+1], attrs={"ENT_TYPE":'GPE'})
                    doc[i]._.grouping_type = "CHILD_PARENT"
                    doc[i]._.list = [child, parent]
            if i+1<len(doc) and doc[i+1].text==',':
                if token.head.dep_=='prep':
                    tail = token
                    while tail.i+2<len(doc) and doc[tail.i+1].text==',' and doc[tail.i+2].ent_type_=='GPE':
                        tail = doc[tail.i+2]
                    if tail!=token:
                        with doc.retokenize() as retokenizer:
                            retokenizer.merge(doc[i:tail.i+1], attrs={"ENT_TYPE":'GPE'})
                        doc[i]._.grouping_type = "CHILD_PARENT"
                        doc[i]._.list = [y.strip() for y in doc[i].text.split(',')]
                else:
                    result = [token]
                    j = 0
                    while j<len(result):
                        result.extend([x for x in result[j].children if x.ent_type_=='GPE'])
                        j+=1
                    if len(result)>1:
                        with doc.retokenize() as retokenizer:
                            retokenizer.merge(doc[i:result[-1].i+1], attrs={"ENT_TYPE":'GPE'})
                        doc[i]._.grouping_type = "CHILD_PARENT"
                        doc[i]._.list = [y.strip() for y in doc[i].text.split(',')]
        elif token.pos_=='NOUN' and token.lemma_.lower() in place_lemmas:
            preps = [child for child in token.children if child.dep_ == 'prep' and child.lemma_.lower() in ['of']]
            for prep in preps:
                children = [x for x in prep.rights if x.pos_=='PROPN']
                if children:
                    parents = [x for x in token.lefts if x.pos_ in ['DET','PROPN','ADJ']]
                    parent = [x for x in parents if x.pos_=='PROPN']
                    if parent:
                        ls = [children[-1].text, parent[0].text]
                        with doc.retokenize() as retokenizer:
                            retokenizer.merge(doc[parents[0].i:children[-1].i+1], attrs={"ENT_TYPE":'GPE','POS':'PROPN'})
                        doc[parents[0].i]._.grouping_type = "CHILD_PARENT"
                        doc[parents[0].i]._.list = ls
                        i = parents[0].i
                    else:
                        parent = [x for x in parents if x.ent_type_=='NORP' and x.head==token]
                        gpe_text = get_gpe_from_norp(parent[0].lemma_.lower()) if parent else None
                        if gpe_text:
                            ls = [children[-1].text, gpe_text]
                            with doc.retokenize() as retokenizer:
                                retokenizer.merge(doc[parents[0].i:children[-1].i+1], attrs={"ENT_TYPE":'GPE','POS':'PROPN'})
                            doc[parents[0].i]._.grouping_type = "CHILD_PARENT"
                            doc[parents[0].i]._.list = ls
                            i = parents[0].i
        i += 1
    return doc

def print_token_attr(doc):
    for token in doc:
        #print(token.text, token.ent_type_, token.ent_iob_, token.lemma_, token.pos_, token.tag_, token.shape_, token.is_alpha, token.is_stop, token.left_edge, token.right_edge, token.ent_kb_id_, token.norm_, token.subtree, token.n_rights, token.conjuncts, token.ancestors, token.head, token.dep_, [x.text for x in token.children])
        #print(token.text, token.lemma_, token.ent_type_, token.pos_, token.left_edge, token.right_edge, token.ent_kb_id_, 
        #      token.norm_, [x for x in token.subtree], token.n_rights, token.conjuncts, [x for x in token.ancestors], 
        #      token.head, token.dep_, [x.text for x in token.children])
        print(token.text, token.ent_type_, token._.list if token._.grouping_type in ['CHILD_PARENT', 'SIBLING'] else '', 
              token.pos_, token.dep_, token.head,[x for x in token.lefts], [x for x in token.rights])

def insert_part_instance(doc, place, in_place):
    print(place, 'in', in_place)
    extraction = {"template":"PART", "sentences":[doc[:].sent.text]}
    arguments = {}
    arguments["1"], arguments["2"] = place, in_place
    extraction["arguments"] = arguments
    result["extraction"].append(extraction)
    
def insert_buy_instance(doc, buyer, item, price, quantity, source):
    print(buyer, 'bought', quantity, item, 'for', price, 'from', source)
    extraction = {"template":"BUY", "sentences":[doc[:].sent.text]}
    arguments = {}
    arguments["1"], arguments["2"], arguments["3"], arguments["4"], arguments["5"] = buyer, item, price, quantity, source
    extraction["arguments"] = arguments
    result["extraction"].append(extraction)
    
def insert_work_instance(doc, person, org, position, location):
    print(person, 'is', position, 'of', org, 'in', location)
    extraction = {"template":"WORK", "sentences":[doc[:].sent.text]}
    arguments = {}
    arguments["1"], arguments["2"], arguments["3"], arguments["4"] = person, org, position, location
    extraction["arguments"] = arguments
    result["extraction"].append(extraction)
    
def insert_ancestor_list(doc, arr):
    if len(arr)>1:
        for x in range(len(arr)-1):
            for y in range(x+1,len(arr)):
                insert_part_instance(doc, arr[x], arr[y])
                
def insert_child_of_list(doc, child, arr):
    for parent in arr:
        insert_part_instance(doc, child, parent)
        
def insert_on_type(doc, child, parent):
    if child._.grouping_type in ['CHILD_PARENT', 'SIBLING']:
        for child_place in child._.list:
            if parent._.grouping_type in ['CHILD_PARENT', 'SIBLING']:
                insert_child_of_list(doc, child_place, parent._.list)
            else:
                insert_part_instance(doc, child_place, parent.text)
    else:
        if parent._.grouping_type in ['CHILD_PARENT', 'SIBLING']:
            insert_child_of_list(doc, child.text, parent._.list)
        else:
            insert_part_instance(doc, child.text, parent.text)
            
def get_gpe_from_norp(word):
    for defn in feature_extraction(word)[0]:
        if defn.startswith('of or relating to'):
            for ent in nlp_wo_pipe(defn).ents:
                if ent.label_=='GPE':
                    return ent.text
    return None

def extract_place_in_place(doc):
    i = 0    
    #print_token_attr(doc)
    while i<len(doc):
        token = doc[i]
        if token.pos_ == 'PROPN':
            if token.ent_type_=='GPE':
                if token._.grouping_type == 'CHILD_PARENT':
                    insert_ancestor_list(doc, token._.list)
            if token.dep_=='nsubj':
                if token.head.lemma_.lower()=='be':
                    nouns = [child for child in token.head.children if child.pos_=='NOUN' and child.lemma_.lower() in place_lemmas]
                    preps = [child for noun in nouns for child in noun.children if child.dep_ == 'prep' and child.lemma_.lower() in ['of','in']]
                    for prep in preps:
                        for child in prep.children:
                            if child.ent_type_=='GPE':
                                insert_on_type(doc, token, child)                  
                            elif child.pos_ == 'PROPN':
                                insert_part_instance(doc, token.text, child.text)
                            elif child.pos_=='NOUN' and child.lemma_.lower() in place_lemmas:
                                for c in child.children:
                                    if c.pos_=='PROPN':
                                        insert_on_type(doc, token, c)
            elif token.dep_=='poss':
                if token.ent_type_=='GPE':
                    if token.head.pos_ == 'PROPN':
                        insert_on_type(doc, token.head, token)
            else:
                pass
        elif token.pos_=='ADP':
            if token.dep_=='prep':
                if token.lemma_.lower()=='in':
                    child = token.head
                    if child.ent_type_ in ['GPE','LOC','LOC DET']:
                        parents = [x for x in token.rights if x.ent_type_=='GPE']
                        if parents:
                            for parent in parents:
                                insert_on_type(doc, child, parent)
                        else:
                            if token.i+1<len(doc) and token.nbor().ent_type_=='GPE':
                                parent = token.nbor()
                                insert_on_type(doc, child, parent)
                    elif child.lemma_.lower() in place_lemmas:
                        child = None
        i += 1
    return doc

def merge_titles(doc):
    i = 0         
    #print_token_attr(doc)
    while i<len(doc):
        token = doc[i]
        if token.ent_type_=='JOBTITLE':
            if len(token.conjuncts)>0:
                tail = token.conjuncts[-1]
                ls = []
                j = token.i
                while j<tail.i+1:
                    x = doc[j]
                    if x.ent_type_=='JOBTITLE':
                        if x.dep_=='amod':
                            arr = [y.text for y in x.head.subtree if y.dep_ in ['amod', 'punct','appos']]
                            ls.append(' '.join(arr))
                            j += len(arr)-1
                        elif x.dep_ == 'subtok' and x.i+2<len(doc) and x.nbor().dep_=='subtok' and x.nbor().nbor().dep_!='subtok':
                            arr = [x.text, x.nbor().text, x.nbor().nbor().text]
                            ls.append(''.join(arr))
                            j += len(arr)-1
                        else:
                            ls.append(x.text.strip())
                    j += 1
                with doc.retokenize() as retokenizer:
                    retokenizer.merge(doc[token.i : tail.i+1], attrs={"ENT_TYPE":'JOBTITLE'})
                doc[token.i]._.grouping_type = "TITLES"
                doc[token.i]._.list = '; '.join(ls)
                #print(doc[token.i], doc[token.i]._.list)
        i += 1
    return doc

def extract_org_title(doc):
    i = 0         
    #print_token_attr(doc)
    while i<len(doc):
        token = doc[i]
        if token.ent_type_ == 'PERSON':
            if token.head.lemma_.lower() in ['be','become']:
                positions = [x for x in token.head.subtree if x.ent_type_ =='JOBTITLE']
                for position in positions:
                    preps = [x for x in position.rights if x.lemma_.lower()=='of']
                    if preps:
                        for prep in preps:
                            orgs = [x for x in prep.rights if x.ent_type_ in ['ORG','GPE']]
                            for org in orgs:
                                insert_work_instance(doc, token.text, org.text, position._.list if position._.grouping_type == "TITLES" else position.text.strip(), '')
                    else:
                        insert_work_instance(doc, token.text, main_ent[filename], position._.list if position._.grouping_type == "TITLES" else position.text.strip(), '')
        elif token.ent_type_ == 'JOBTITLE':
            if token.dep_ == 'compound':
                if token.head.ent_type_=='PERSON':
                    insert_work_instance(doc, token.head.text, main_ent[filename], token._.list if token._.grouping_type == "TITLES" else token.text.strip(), '')
        elif token.pos_=='PROPN':
            if token.dep_ == 'ROOT':
                be = [x for x in token.rights if x.lemma_.lower() in ['be','become']]
                if be:
                    for b in be:
                        title = [x for x in b.rights if x.ent_type_ =='JOBTITLE']
                        if title:
                            insert_work_instance(doc, token.text, main_ent[filename], title[0]._.list if title[0]._.grouping_type == "TITLES" else title[0].text.strip(), '')
            else:
                if token.head.lemma_.lower() in ['be','become']:
                    title = [x for x in token.head.rights if x.ent_type_ =='JOBTITLE']
                    if title:
                        insert_work_instance(doc, token.text, main_ent[filename], title[0]._.list if title[0]._.grouping_type == "TITLES" else title[0].text.strip(), '')
        i += 1
    return doc

#Praveen
def extract_all_named_entities(doc):
    template = {"buyer":"","item":"","price":"","quantity":"","source":""}
    entities = [ent for ent in doc.ents if ent.label_ in ["BUY","MONEY","ORG","PERSON","PRODUCT","PERCENT","CARDINAL"]]
    print("Entities",entities)
    list_of_templates = []
   
    #print("Code in first rule: (X acquired/purchased Y for dollars)")
    #print("**************************************")
    def constraint_check(entities):
        list_t = []
        if len(entities)==3:
            template = {"buyer":"","item":"","price":"","quantity":"","source":""}
            for ent in ["acquire","purchase","buy","brought","purchased","acquired"]:
                    if ent.strip()==entities[1].text:
                        template["buyer"]= entities[0].text
                        template["item"]= entities[2].text
            for ent in ["acquired by","purchased by"]:
                    if ent.strip()==entities[1].text:
                        template["buyer"]= entities[2].text
                        template["item"] = entities[0].text
            if len(template["buyer"])>0 and len(template["item"])>0:
                list_t.append(template)
        else:
            for index in range(len(entities)):
                template = {"buyer":"","item":"","price":"","quantity":"","source":""}
                for ent in ["acquire","purchase","buy","brought","purchased","acquired","acquisition"]:
                    if (entities[index].text.strip()==ent.strip()):
                        if index+2<len(entities):
                            if entities[index-1].label_  in ["ORG","PERSON","PRODUCT"] and entities[index+1].label_ in ["ORG","PERSON","PRODUCT"] and entities[index+2].label_!="MONEY":
                                template["buyer"]= entities[index-1].text
                                template["item"] = entities[index+1].text
                        else:
                            if index-1>0 and index+1<len(entities):
                                if entities[index-1].label_  in ["ORG","PERSON","PRODUCT"] and entities[index+1].label_ in ["ORG","PERSON","PRODUCT"]:
                                    template["buyer"]= entities[index-1].text
                                    template["item"] = entities[index+1].text
                           
                if len(template["buyer"])>0 and len(template["item"])>0:
                    list_t.append(template)  
                for ent in ["acquired by","purchased by"]:
                    if entities[index].text.strip()==ent and index-1 and index+1:
                        if index+2<len(entities):
                            if entities[index-1].label_  in ["ORG","PERSON","PRODUCT"] and entities[index+1].label_ in ["ORG","PERSON","PRODUCT"] and entities[index+2].label_!="MONEY":
                                template["buyer"]= entities[index-1].text
                                template["item"] = entities[index+1].text
                        else:
                            if entities[index-1].label_  in ["ORG","PERSON","PRODUCT"] and entities[index+1].label_ in ["ORG","PERSON","PRODUCT"]:
                                template["buyer"]= entities[index-1].text
                                template["item"] = entities[index+1].text
                if len(template["buyer"])>0 and len(template["item"])>0:
                    list_t.append(template)
        return list_t

    if len(entities)>=3:
        list_of_templates = constraint_check(entities)
    if len(template["buyer"])==0:
        for ent in entities:      
                head = ent.root.head
                for token in head.children:
                    if (token.pos_ in ["PROPN","NOUN"] and token.dep_=="nsubj" and token.ent_type_ in ["ORG","PERSON","PRODUCT"] and token.head.pos_=="VERB"):
                        template["buyer"] = token.text
                    if (token.pos_=="PROPN" and token.dep_=="nsubj" and token.head.pos_=="VERB" and token.head.dep_=="ROOT"):
                        template["buyer"] = token.text
                    if (token.pos_=="PROPN" and token.dep_=="appos" and token.head.pos_=="NOUN" and token.head.dep_!="dobj"):
                        template["buyer"] = token.text
                    if (token.pos_=="VERB" and token.dep_=="relcl" and token.head.pos_=="PROPN"):
                        template["buyer"] = token.head.text
                    if (token.pos_ in ["PROPN","NOUN"] and  token.dep_ =="dobj" and token.head.pos_=="VERB" and token.head.ent_type_=="BUY"):
                        template["item"] = token.text
                    if (len(template["item"])!=0 and token.pos_ in ["PROPN","NOUN","ADV"] and token.dep_ in ["dobj","pobj","nsubjpass"] and token.head.pos_=="VERB" and token.ent_type_=="BUY" and token.head.dep_!="advcl"):
                        template["item"] = token.text
                    if (len(template["item"])!=0 and token.pos_=="PROPN" and token.dep_=="dobj" and token.head.ent_type_=="BUY" and token.ent_type_ in ["ORG","PERSON","PRODUCT"]):
                        template["item"] = token.text
                    if (len(template["item"])!=0 and token.pos_=="PROPN" and token.dep_=="appos" and token.head.pos_=="NOUN" and token.head.head.ent_type_=="BUY"):
                        template["item"] = token.text
                    if (token.pos_=="PROPN" and token.dep_=="compound" and token.head.pos_=="PROPN"):
                        template["item"] = token.text
                    if (token.pos_=="NUM" and token.dep_ in ["npadvmod","pobj","nummod"] and token.ent_type_ in ["MONEY","CARDINAL"]):
                        if token.head.pos_=="NOUN":
                            template["price"] = str(token.text)+" "+str(token.head.text)
                        else:
                            template["price"] = token.text
                    if (token.pos_ in ["PROPN","ADP"] and token.dep_=="pobj" and  token.head.pos_ in ["ADP","SCONJ"] and token.head.dep_=="prep" and token.head.head.pos_ in ["VERB"] and token.ent_type_ in ["ORG","PERSON","PRODUCT","GPE"]):
                         template["source"] = token.text
                    if (token.ent_type_=="PERCENT" and token.pos_=="NOUN" and token.dep_ in ["compound","dobj","npadvmod","nmod"]):
                        template["quantity"] = token.text
                    #print(template)
        if len(template["buyer"])>0 and len(template["item"])>0:
            list_of_templates.append(template)
        #print("first_rule",template)  
        template = {"buyer":"","item":"","price":"","quantity":"","source":""}
        #print("Code in second rule:(X  was acquired/purchased Y for dollars)")
        #print("**************************************")
        for ent in entities:
                head = ent.root.head    
                for token in head.children:
                    if (token.pos_=="VERB" and token.dep_=="acl" and token.ent_type_ in ["BUY"]):
                        template["item"]=head.text
                    if (token.pos_=="PROPN" and token.dep_=="nsubjpass" and  token.ent_type_ in ["ORG","PERSON","PRODUCT"]):
                            template["item"] = token.text
                    if (token.pos_=="PROPN" and token.dep_=="pobj") and (token.ent_type_ in ["ORG","PERSON","PRODUCT"]):
                            template["buyer"] = token.text
                    if (token.ent_type_=="MONEY" and token.pos_=="NUM" and token.dep_ in ["npadvmod","pobj","nummod"]):
                            template["price"] = token.text
                    if (token.ent_type_=="PERCENT" and token.pos_=="NOUN" and token.dep_ in ["compound","dobj","npadvmod"]):
                        template["quantity"] = token.text
        if len(template["buyer"])>0 and len(template["item"])>0:
            list_of_templates.append(template)
            template = {"buyer":"","item":"","price":"","quantity":"","source":""}
    #print("second_rule",template)
    list_of_templates= [dict(y) for y in set(tuple(x.items()) for x in list_of_templates)]
    
    print("Final Template:",list_of_templates)
    for template in list_of_templates:
        insert_buy_instance(doc, template['buyer'], 
                            template['item'], 
                            template['price'], 
                            template['quantity'], 
                            template['source'])
    return doc
#Praveen

if __name__=='__main__':
    s = time.time()
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    nlp = spacy.load('en_core_web_sm')
    nlp_wo_pipe = spacy.load('en_core_web_sm')
    matched_sents = []  # Collect data of matched sentences to be visualized
    #filename = '4.txt'
    filename = None
    dataset = load_text_files(filename)
    prep = Preprocess()
    Token.set_extension("grouping_type", default=None, force=True)
    Token.set_extension("list", default=None, force=True)
    place_lemmas = ['place', 'village', 'county', 'city', 'district', 'region', 'state', 'capital', 'country', 'continent', 'world']
    #print(feature_extraction('in'))
    
    new_dataset = {}
    #new_dataset = dataset
    
    main_ent = {}
    for filename in dataset:
        if len(dataset[filename])>0:
            paragraph = dataset[filename][0]
            paragraph =  prep.remove_unwanted_character(paragraph).lstrip()
            doc = nlp(paragraph)
            main_ent[filename] = [ent for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'GPE']][0].text
    print('main ent', main_ent)
    coref = neuralcoref.NeuralCoref(nlp.vocab)
    nlp.add_pipe(coref, name='neuralcoref', after='ner')
    for filename in dataset:
        new_dataset[filename] = []
        main_ent_replaced = False
        false_ent = ''
        true_ent = ''
        for k, paragraph in enumerate(dataset[filename]):
            paragraph =  prep.remove_unwanted_character(paragraph).lstrip()
            doc = nlp(paragraph)
            if k==0:
                if doc._.coref_clusters[0][0].text.startswith(main_ent[filename]):
                    main_ent_replaced = True
                    false_ent = doc._.coref_clusters[0].main.text
                    true_ent = main_ent[filename]
            new_dataset[filename].append(doc._.coref_resolved.replace(false_ent, true_ent) if main_ent_replaced else doc._.coref_resolved)
    nlp.remove_pipe("neuralcoref")
    
    nlp.add_pipe(merge_ents, name='merge_ents', after='ner')
    nlp.add_pipe(merge_siblings, name='merge_siblings', after='merge_ents')
    nlp.add_pipe(merge_ancestors, name='merge_ancestors', after='merge_siblings')
    nlp.add_pipe(extract_place_in_place, name='extract_place_in_place', after='merge_ancestors')
    
    #Praveen
    from spacy.pipeline import EntityRuler
    definition,synonyms,antonyms,hyper,hypo,mero,holo = feature_extraction('acquired')            
    buy_ruler = EntityRuler(nlp)
    buy_patterns = [{"label": "BUY", "pattern": "purchase"},{"label": "BUY", "pattern": "acquisition"},{"label": "BUY", "pattern": "buy"},{"label": "BUY", "pattern": "acquire"},{"label": "BUY", "pattern": "purchased"},{"label": "BUY", "pattern": "purchased by"},{"label": "BUY", "pattern": "acquired by"},{"label": "BUY", "pattern": "brought"},{"label": "BUY", "pattern": "acquired"}]
    buy_ruler.add_patterns(buy_patterns)
    nlp.add_pipe(buy_ruler, name='buy_ruler', after='extract_place_in_place')
    title_ruler = EntityRuler(nlp)
    title_patterns = pickle.load(open("title_patterns.pkl", "rb"))
    title_ruler.add_patterns(title_patterns)
    nlp.add_pipe(title_ruler, name='title_ruler', after='buy_ruler')
    nlp.add_pipe(merge_titles, name='merge_titles', after='title_ruler')
    nlp.add_pipe(extract_org_title, name='extract_org_title', after='merge_titles')
    #Praveen
    
    for filename in new_dataset:
        result  = {"document":filename, "extraction":[]}
        with open('dataset/'+filename, "w") as outfile: 
            outfile.write('')
        for paragraph in new_dataset[filename]:
            with open('dataset/'+filename, "a") as outfile: 
                outfile.write(paragraph)
                outfile.write('\n')
            paragraph = prep.sentence_tokenize(paragraph)
            for each_sent in paragraph:
                sent_bound = []
                for l in range(len(each_sent)):
                    if each_sent[l]=='.' and l+1<len(each_sent) and each_sent[l+1].isupper():
                        sent_bound.append(l+1)
                for m in sent_bound:
                    each_sent = each_sent[:m] + ' ' + each_sent[m:]
                lem_sent = prep.lemmatize(each_sent)
                cleaned_sentence = prep.remove_stopwords(lem_sent)
                cleaned_sentence = prep.word_tokenize(cleaned_sentence)
                cleaned_sentence = prep.pos_tag(cleaned_sentence)
                sentence = nlp(each_sent)
                
                #Praveen
                sentence = merge_ents(sentence)
                print(extract_all_named_entities(sentence))
                #for ent in sentence.ents:
                    #print(ent.text,"-->", ent.label_)
                try:
                    displacy.render(sentence, style="ent",jupyter=True)
                    displacy.render(sentence, style="dep",jupyter=True)
                except:
                    pass
                #Praveen
                
        json_object = json.dumps(result, indent = 4)
        #print(json_object)
        with open(filename[:-4]+".json", "w") as outfile: 
            outfile.write(json_object)

    nlp.remove_pipe("merge_ents")
    nlp.remove_pipe("merge_siblings")
    nlp.remove_pipe("merge_ancestors")
    nlp.remove_pipe("extract_place_in_place")
    nlp.remove_pipe("buy_ruler")
    nlp.remove_pipe("title_ruler")
    nlp.remove_pipe("extract_org_title")
    print(time.time()-s)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shari\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shari\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\shari\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


main ent {'1.txt': 'Amazon', '2.txt': 'Steven Paul Jobs', '3.txt': 'John Fitzgerald Kennedy', '4.txt': 'Netflix', '5.txt': 'William Henry Gates III'}
Entities [Amazon, acquired, Whole Foods Market, 13.4 billion]
Final Template: [{'buyer': 'Amazon', 'item': 'Whole Foods Market', 'price': '', 'quantity': '', 'source': ''}, {'buyer': 'Amazon', 'item': 'Whole Foods Market', 'price': '13.4 billion', 'quantity': '', 'source': ''}]
Amazon bought  Whole Foods Market for  from 
Amazon bought  Whole Foods Market for 13.4 billion from 
In 2017, Amazon acquired Whole Foods Market for US 13.4 billion.


Jeff Bezos is CEO of Amazon in 
Entities [Jeff Bezos, Amazon]
Final Template: []
Jeff Bezos is the CEO of Amazon.


Bellevue in Washington
Entities [Amazon, Jeff Bezos]
Final Template: [{'buyer': 'Jeff Bezos', 'item': 'Amazon', 'price': '', 'quantity': '', 'source': ''}]
Jeff Bezos bought  Amazon for  from 
Amazon was founded by Jeff Bezos in Bellevue, Washington, in July 1994.


Entities [Steven Paul Jobs]
Final Template: []
Steven Paul Jobs (; February 24, 1955 - October 5, 2011) was an American business magnate and investor.


...

Entities [brought]
Final Template: []
No charges were brought.


Entities [Microsoft, acquired, the New York Evening News, 50.2 million]
Final Template: [{'buyer': 'Microsoft', 'item': 'the New York Evening News', 'price': '', 'quantity': '', 'source': ''}]
Microsoft bought  the New York Evening News for  from 
In 1977, Microsoft indirectly acquired the New York Evening News for an amount of 50.2 million.


Entities [Microsoft, the Buffalo Courier-Express]
Final Template: []
Antitrust charges started, instigated by Microsoft rival, the Buffalo Courier-Express.


Entities [Microsoft, the Buffalo Courier-Express, the Courier-Express]
Final Template: []
Microsoft rival, the Buffalo Courier-Express lost money, until the Courier-Express folded in 1982.


3.2552592754364014


In [12]:
import spacy
nlp = spacy.load('en_core_web_sm')
sentence = "Silk is the largest export of the Indian city of Jaipur."
doc = nlp(sentence)
for token in doc:
    print(token.lemma_, token.ent_type_, token.pos_, token.dep_, token.head,[x for x in token.lefts], [x for x in token.rights])

silk  NOUN nsubj is [] []
be  VERB ROOT is [Silk] [export, .]
the  DET det export [] []
large  ADJ amod export [] []
export  NOUN attr is [the, largest] [of]
of  ADP prep export [] [city]
the  DET det city [] []
indian NORP ADJ amod city [] []
city  NOUN pobj of [the, Indian] [of]
of  ADP prep city [] [Jaipur]
Jaipur GPE PROPN pobj of [] []
.  PUNCT punct is [] []


In [22]:
dic[0] = ['MS']
print(dic)

[['MS'], Gates: [Gates, Gates], its rival, the Buffalo Courier-Express: [its rival, the Buffalo Courier-Express, Both papers]]


In [3]:
"""
1. Where GPE is incorrect : millennia
2. Where GPE is incorrect : It offers a selection of more than 40,000 comic books and graphic novels across Android, iOS PLACE_IN_PLACE , Fire OS and Windows 8 devices and over a web browser.
3. Show actual sentence with all unwanted characters : In 1819, the Adams-Onís Treaty between the United States and Spain defined the Red River as the northern boundary of New Spain, officially placing the future location of Dallas well within Spanish territory.
4. NORP entities like Indian
5. Dependency is incorrect : Along with Brazil, Russia, India and South Africa, China is a member of the BRICS group of emerging major economies and hosted the group's third official summit at Sanya, Hainan in April 2011.
                            China has over 160 cities with a population of over one million, including the seven megacities (cities with a population of over 10 million) of Chongqing, Shanghai, Beijing, Guangzhou, Tianjin, Shenzhen, and Wuhan.
                            Guizhou, one of the poorest provinces in China, only totalled 3,204.
6. Include LOC in GPE's
7. Include FAC in GPE's
8. Semantic inclusion of places : China is home to many of the world's tallest religious statues, including the tallest of all, the Spring Temple Buddha in Henan.
                                    India contains Delhi and Mumbai.
                                    India includes Delhi and Mumbai.
9. In 2011, Shenzhen in Guangdong, China hosted the 2011 Summer Universiade.
10. Time Magazine named The 14th Dalai Lama, Lech Wałęsa, Martin Luther King, Cesar Chavez, Aung San Suu Kyi, Benigno Aquino, Jr., Desmond Tutu, and Nelson Mandela as Children of Gandhi and his spiritual heirs to nonviolence.The Mahatma Gandhi District in Houston, Texas, United States, an ethnic Indian enclave, is officially named after Gandhi.

"""

"\n1. Where GPE is incorrect : millennia\n2. Where GPE is incorrect : It offers a selection of more than 40,000 comic books and graphic novels across Android, iOS PLACE_IN_PLACE , Fire OS and Windows 8 devices and over a web browser.\n3. Show actual sentence with all unwanted characters : In 1819, the Adams-Onís Treaty between the United States and Spain defined the Red River as the northern boundary of New Spain, officially placing the future location of Dallas well within Spanish territory.\n4. NORP entities like Indian\n5. Dependency is incorrect : Along with Brazil, Russia, India and South Africa, China is a member of the BRICS group of emerging major economies and hosted the group's third official summit at Sanya, Hainan in April 2011.\n                            China has over 160 cities with a population of over one million, including the seven megacities (cities with a population of over 10 million) of Chongqing, Shanghai, Beijing, Guangzhou, Tianjin, Shenzhen, and Wuhan.\n   

In [62]:
''.join(['a','b','c'])

'abc'